<a href="https://colab.research.google.com/github/Z3376/Sentence-Similarity-with-BERT/blob/master/Sentence_similarity_py2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('./gdrive')

In [2]:
!pip install pdfminer

     |████████████████████████████████| 4.1MB 6.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/1b/af/91/b925461baf990ee92513dd451237a7570fca3e7f8dd5439e5b
Successfully built pdfminer


In [3]:
!pip install PyPDF

  Using cached https://files.pythonhosted.org/packages/bd/22/e1df75dffb7679344bcf986abd473d7c0e22ba976f2caef31551e394a3a2/pyPdf-1.13.tar.gz
  Stored in directory: /root/.cache/pip/wheels/e9/be/bd/9953d15f2e762c0592d9c3e2894ac717570aaff7f088654dba
Successfully built PyPDF


In [83]:
!pip install keras_bert

  Stored in directory: /root/.cache/pip/wheels/6c/63/15/9755f6e729fc47e95b56cea2273a8c49435fce7ecdfed4d1e5
  Stored in directory: /root/.cache/pip/wheels/d5/77/37/8fcad8efc82751342b5e19134629444394a316cee7e0955f4d
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Stored in directory: /root/.cache/pip/wheels/95/49/02/4eda210bc4c37ff1d45311665bceb790881dbea92b27b025a5
  Stored in directory: /root/.cache/pip/wheels/c3/9b/9e/f4072915f660e90bb3638332276f4de80476f3afcb5d010d6f
  Stored in directory: /root/.cache/pip/wheels/39/e2/e2/3514fef126a00574b13bc0b9e23891800158df3a3c19c96e3b
  Stored in directory: /root/.cache/pip/wheels/d1/bc/b1/b0c45cee4ca2e6c86586b0218ffafe7f0703c6d07fdf049866
  Stored in directory: /root/.cache/pip/wheels/cc/dc/17/84258b27a04cd38ac91998abe148203720ca696186635db694
Successfully built keras-bert keras-transformer keras-pos-embd keras-multi-head keras-layer-normalization keras-position-wise-feed-forward keras

In [0]:
# import pyPdf as pr
import numpy as np
import re

In [0]:
from cStringIO import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt

In [0]:
import keras_bert
import keras
import codecs

# Data

In [0]:
file1 = './gdrive/My Drive/Radicali_task/Personal Data Protection Act 2012.pdf'
file2 = './gdrive/My Drive/Radicali_task/Personal Data Protection Regulations 2014.pdf'

In [0]:
# def pdf_text(file,page_num):
#     pdf_file = open(file,'rb')
#     pdf_reader = pr.PdfFileReader(pdf_file)
#     page = pdf_reader.getPage(page_num)
#     text = page.extractText().strip().split('.')
#     return text

In [0]:
def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = file(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 

In [0]:
text1 = convert(file1).strip().split('\n')
text2 = convert(file2).strip().split('\n')

In [0]:
text1_ = [unicode(' '.join(re.sub('[^a-zA-Z0-9]',' ',text1[i]).split()).lower()) for i in range(len(text1))]
text2_ = [unicode(' '.join(re.sub('[^a-zA-Z0-9]',' ',text2[i]).split()).lower()) for i in range(len(text2))]

# Embedding

In [0]:
pretrained_path = './gdrive/My Drive/uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path,'bert_config.json')
ckpt_path = os.path.join(pretrained_path,'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path,'vocab.txt')

In [0]:
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
tokenizer = keras_bert.Tokenizer(token_dict)

In [0]:
def embed(sen1,corpus,tokenizer,max_len=128):
    ind_array = []
    seg_array = []
    for i in range(len(corpus)):
        ind,seg = tokenizer.encode(sen1,corpus[i],max_len=max_len)
        ind_array.append(ind)
        seg_array.append(seg)
    ind_array = np.array(ind_array)
    seg_array = np.array(seg_array)
    return ind_array,seg_array

# Model

In [0]:
bert = keras_bert.load_trained_model_from_checkpoint(config_path,ckpt_path,training=True,trainable=True,seq_len=128)

In [0]:
ind,seg = bert.input[:2]
outpt = bert.get_layer('NSP').output
model = keras.models.Model([ind,seg],outpt)

In [0]:
# model.summary()

# Inference

In [0]:
test = 291     

In [0]:
ind,seg = embed(text1_[test],text2_,tokenizer)

In [0]:
pred = model.predict([ind,seg])

In [0]:
result = np.argmax(pred[:,0])

In [214]:
text1_[test]

u'organisations and to establish do not call register and to provide'

In [215]:
text2_[result]

u'organisation to respond to the applicant s request'